# Datetime in Pandas

Wenn man `datetime`-Daten mit Pandas einliest, werden diese nicht umgewandelt und standardmäßig nur als String gespeichert. Wenn wir mit Datumsangaben rechnen wollen, müssen wir wie in Standard-Python mit dem Modul datetime die Werte innerhalb von Pandas in einen Datetime-Typen umwandeln.

ACHTUNG: Pandas ist nicht mit dem Standard `datetime` Modul kompatibel, sondern benutzt die Typen `numpy.datetime64` und `numpy.timedelta64`.

In [ ]:
from datetime import date
import numpy as np
import pandas as pd
import seaborn as sns

## Datetime-Serien erstellen

In [ ]:
# Versuchen datetime.date in Pandas zu integrieren
dates = [date(2023, 10, x) for x in range(1, 11)]
dates

In [ ]:
# datetime.date Objekte können in eine Series eingebaut werden.
# Aber: Das ist nicht der Standarddatentyp in Pandas!
# Weil der Datentyp nicht passt, können wir viele sehr nützliche Operationen, 
# die es in Pandas gibt, gar nicht benutzen, etwa Zeitdaten aggregieren (Details später).
date_series = pd.Series(dates, name='dates')
date_series

In [ ]:
# Umwandeln der Zeilen in den Standarddatentypen
# von Pandas für Zeitdaten:
date_series_dt64 = date_series.astype("datetime64[ns]")

In [ ]:
# Dieser, von numpy stammende Datentyp, sieht gleich aus, kann aber 
# innerhalb von Pandas viel mehr!
date_series_dt64

In [ ]:
# Man kann mit Pandas auch Datums-Reihen selbst erstellen,
# die dann sofort im richtigen Datentypen vorliegen:
dates = pd.date_range("2023-10-01", "2023-10-31")
dates

In [ ]:
# Mit freq kann man auch die Schrittweite steuern, also z.B. jeder zweite Tag nur:
every_second_day = pd.date_range(start="2023-10-01", 
                                 end="2023-10-31",
                                 freq='2D'
                                 )
every_second_day

In [ ]:
# Die Schrittweite beträgt im folgenden Beispiel 'ME' für monatlich und am Monatsende
every_month = pd.date_range(start="2023-10",
                            end='2024-02',
                            freq='ME',
                            )
every_month

In [ ]:
# Mit 'MS' (S für Start) kriegt man dagegen alle Monate mit erstem Tag!
every_month = pd.date_range(start="2023-10",
                            end='2024-02',
                            freq='MS',
                            )
every_month

In [ ]:
# Man braucht nicht zwingend start und end!
# Es ist auch möglich ein Start-Datum anzugeben und dann zu sagen,
# wie viele Einträge der Index umfassen soll (periods, hier 12)!
periods = pd.date_range(start="2023-10-01",
                        freq='MS',
                        periods=12
                        )
periods

# Rechnen mit Datetime


In [ ]:
# Vergangene Tage seit einem bestimmten Datum berechnen
np.datetime64("today") - np.datetime64("2000-01-01")

In [ ]:
# Jetzt holen wir wieder unseren Datetime-Index und rechnen damit!
dates = pd.date_range("2023-10-01", "2023-10-31")
dates

In [ ]:
# Wie aus Numpy bereits gewohnt, wird das Einzeldatum Wert für Wert
# von allen Daten im DatetimeIndex subtrahiert und es entsteht ein TimedeltaIndex:
dates - np.datetime64("2000-01-01")

In [ ]:
# Man kann auch Timedelta-Objekte erstellen und die Daten um dieses Delta verschieben,
# etwa wie hier um 31 Tage in die Zukunft:
np.timedelta64(31, "D") + dates

## Einzelkomponenten von Datumsangaben

In [ ]:
dates

In [ ]:
# Monat des jeweiligen Datums
dates.month

In [ ]:
# Wochentag (Zahl) des jeweiligen Datums
dates.weekday

In [ ]:
# Wochentag (Wort) des jeweiligen Datums
dates.day_name()

## Strings in datetime umwandeln

Mit `pandas.to_datetime` können Werte in den Pandas-datetime-Typen (pd.Timestamp) konvertiert werden.

In [ ]:
# Umwandeln eines Strings
pd.to_datetime("2024-02-05", format="%Y-%m-%d")

In [ ]:
# Umwandeln eines anders gearteten Strings
pd.to_datetime("feb 24 05", format="%b %y %d")

In [ ]:
# Datentyp prüfen:
my_date = pd.to_datetime("feb 24 05", format="%b %y %d")
print(type(my_date))

In [ ]:
# Format muss man zwar nicht unbedingt angeben, denn
# es gibt einen automatischen Parser, der oftmals funktioniert...
pd.to_datetime("feb 2024 05")

In [ ]:
# aber auch gerne falsche Daten extrahiert!
pd.to_datetime("feb 24 05")
# Ratschlag: Auf den automatischen Parser verzichten.

In [ ]:
# Natürlich akzeptiert to_datetime auch date-Objekte:
pd.to_datetime(date(2023, 10, 26))

In [ ]:
# DataFrame: Strings in Datetime umwandeln
df = pd.DataFrame({
    "date": ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
             '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
             '2022-01-09', '2022-01-10'],
    "temperatur": 10 + np.random.randint(10, size=10) 
})
df

In [ ]:
# Datentypen in den Spalten prüfen (date: object heißt, dass es Strings sind):
df.dtypes

In [ ]:
# Umwandeln in Pandas datetime:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
# dtype der Datums-Spalte nachher
df.dtypes

In [ ]:
# Es wäre cool, auf die verschiedenen Bestandteile der Datumsobjekte zugreifen zu können.
# Zum Beispiel day oder day_of_week oder year etc.
# Ein Weg, das zu erreichen: hinter den Spaltennamen ein dt und dann das gewünschte Attribut anhängen.
# Hier Wochentage als Zahlen (0 = Montag, 6 = Sonntag):
df['date'].dt.dayofweek

In [ ]:
# Name der Wochentage? Auch kein Problem mit der Methode day_name()
df['date'].dt.day_name()

In [ ]:
# Ein anderer, gängiger Weg ist es, die Datumsspalte gleich zum Index zu erheben,
# was die Arbeit mit ihr weiter erleichtert:
df.set_index('date', inplace=True)

In [ ]:
df.index.year

In [ ]:
df.index.dayofweek

In [ ]:
df.index.day_name()

##### Übungsaufgabe

Errechne für den kompletten Datensatz das Alter der Kinder und lasse dir im Anschluss das Minimum, Maximum, und den Mittelwert von Alter, Größe und Gewicht anzeigen. Gehe der Einfachheit halber von einem Jahr aus, das aus 365 Tagen besteht, denn für genauere Altersberechnungen fehlen uns noch ein paar Funktionalitäten. (Interessanter [Link](https://www.geeksforgeeks.org/convert-birth-date-to-age-in-pandas/) zum Thema)

In [ ]:
schule_df = pd.DataFrame({
    'SchulID': ['s001','s002','s003','s001','s002','s004'],
    'Klasse': ['V', 'V', 'VI', 'VI', 'V', 'VI'],
    'Name': ['Alberto Franco','Gino Mcneill','Ryan Parkes', 'Eesha Hinton', 'Gino Mcneill', 'David Parkes'],
    'Geburtsdatum': ["2002-05-15","2002-05-17","1999-01-16","1998-09-25","2002-05-11","1997-09-15"],
    'Groesse': [173, 192, 186, 167, 151, 159],
    'Gewicht': [35, 32, 33, 30, 31, 32]},
    index=['S1', 'S2', 'S3', 'S4', 'S5', 'S6'])

schule_df

### Exkurs: mit apply() lassen sich Funktionen auf ganze Spalten anwenden!

In [ ]:
# Wir wollen eine ganz einfache Lambda-Funktion auf Alter anwenden, die aus jedem Float eine Ganzzahl macht:
schule_df['Alter'].apply(lambda x: int(x))

In [ ]:
# Wir wollen cm in feet umrechnen mit einer Funktion:
# 1cm ist dabei circa 0,033 Feet groß
def cm_to_feet(size_cm: float) -> float:
    return size_cm * 0.033

In [ ]:
# Jetzt wenden wir die Funktion mit apply auf die Spalte Groesse an:
schule_df['Groesse'].apply(cm_to_feet)

In [ ]:
# Jetzt wollen wir das Ergebnis als separate Spalte 'Groesse_feet' speichern:
schule_df['Groesse_feet'] = schule_df['Groesse'].apply(cm_to_feet)

In [ ]:
schule_df.head()

In [ ]:
# Aufgabe: Schreibe eine weight_to_pounds oder arbeite mit lambda, um eine neue Spalte
# im Dataframe anzulegen, in der die Gewichtsangaben in Pfund zu finden sind!

## Pandas datetimes in Strings umwandeln

Wir kennen schon `datetime.date.strftime()`. In Pandas können wir die gleiche Funktionalität übernehmen!

In [ ]:
# Wie sehen unsere Daten nochmal aus?
dates

In [ ]:
# Ausgabe von Teilen des Datums mit strftime
dates.strftime("%Y %b")

### Übungsaufgabe 
Stelle die Geburtstage der Kinder in folgendem Format dar: "May 15" (Beispiel bei Alberto Franco) und speichere die formatierten Datumsangaben in einer neuen Spalte namens Geburtstag.

# Datetime beim Einlesen von Dateien

Auch wenn wir CSV- oder Excel-Dateien laden, werden Datumsangaben nicht automatisch ausgelesen, sondern als pure Strings importiert. Daher müssen wir auch hier den Datentyp umwandeln bzw. alternativ beim Import direkt die Zeitdaten parsen.

In [ ]:
# Wir speichern unser Schul-Dataframe als csv:
schule_df.to_csv('schule.csv', index=True)

In [ ]:
# Wir laden dieses erneut als Dataframe:
schule = pd.read_csv('schule.csv', index_col=0)

In [ ]:
# Geburtsdatum ist object (also String):
schule.dtypes

In [ ]:
# Lösung 1: Nachträglich to_datetime anwenden.
schule["Geburtsdatum"] = pd.to_datetime(schule["Geburtsdatum"])
schule.dtypes
# Nachteil: Ein zusätzlicher Arbeitsschritt nötig!

In [ ]:
schule.head()

In [ ]:
# Lösung 2: Das parse_dates keyword Argument beim
# Einlesen verwenden:
schule = pd.read_csv('schule.csv', 
                     index_col=0,
                     parse_dates=['Geburtsdatum'],
                     date_format='%Y-%m-%d')
schule.dtypes
# Achtung: Ohne date_format könnten Daten beim automatischen Parsen falsch erkannt werden!

# Arbeit mit datetime-Indizes

Wenn wir Datetime-Werte als Index einer Series oder eines DataFrames haben, können wir sehr schön damit arbeiten.

So können wir z.B. Slicing benutzen und auch die Frequenz ändern (z.B. tägliche Werte zu monatlichen zusammenfassen).

In [ ]:
sea_ice = sns.load_dataset('seaice').set_index('Date')
sea_ice.head()

## Slicing

In [ ]:
# Abruf einer Zeile
sea_ice.loc["1980-01-01"]

In [ ]:
# Abruf von Daten eines Monats:
sea_ice.loc["1980-02"]

In [ ]:
# Abruf eines Datumsbereichs
# Hier: Alle Daten von Januar bis einschließlich Februar:
sea_ice.loc["1980-01":"1980-02"]

In [ ]:
# Nach Slicing kann man weiterhin auf Datumsbestandteile
# zugreifen. Hier bei nur einem Eintrag:
sea_ice.loc["1980-01-01"].name.month

In [ ]:
# Hier bei einem Array aus Einträgen:
sea_ice.loc["1980-01":"1980-03"].index.month

## Resampling

Wenn wir datetime als Index benutzen, können wir die "Frequenz" der Zeitdaten ändern.
Frequenz bezeichnet die Feinkörnigkeit der Daten: gibt es zu jedem Tag einen Eintrag, zu jedem Monat, zu jedem Jahr?
Man kann mit dem Frequenz-Parameter Zeitdaten entsprechend umwandeln.

<br>ACHTUNG: Wenn man eine niedrigere Frequenz wählt (Also zum Beispiel tägliche Daten zu monatlichen Daten zusammenfasst), hat man in der Regel mehrere Werte pro Index. Diese müssen dann noch mit einer Aggregations-Funktion zusammengefasst werden (vergleichbar mit Groupby!).

Wenn man andererseits eine höhere Frequenz wählt, also zum Beispiel monatliche Daten zu täglichen Daten umwandelt, erhält man viele fehlende Werte. Wenn es gute Gründe dafür gibt, können diese künstlich aufgefüllt werden. Ansonsten ist besser, auf dem Level der Granularität (Körnigkeit) der Daten zu bleiben, auf dem diese vorliegen.

In [ ]:
# Aktueller DataFrame
sea_ice

In [ ]:
# Zusammenfassung aller Daten eines Jahres zum Durchschnitt:
sea_ice.resample("1Y").mean()

In [ ]:
# Plotten
sea_year = sea_ice.resample("1Y").mean()
sea_year.index = sea_year.index.year
sea_year.plot(marker='o',
              ylabel='Temp (°C)');

Übersicht über die rule Argumente von resample: [Link](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)

Unter der Überschrift "DateOffset objects"

In [ ]:
# Zusammenfassung der Daten für 10 Jahre im Durchschnitt:
sea_ice.resample("10Y").mean()

In [ ]:
# Daten zum Monat zusammenfassen
sea_ice_month = sea_ice.resample("MS").mean()
sea_ice_month

In [ ]:
# Auffüllen der Tage mit linearer Interpolation
# Standardwert ist 'linear', aber es gibt noch andere.
sea_ice_month.resample("D").interpolate()

In [ ]:
sea_ice_month.resample("D").interpolate(method='linear')

In [ ]:
# Alternative Variante: Mit asfreq tägliche Daten erzeugen, in denen auch NAs vorliegen:
sea_ice.asfreq('1D')

In [ ]:
# Hier kann man auch Methoden zum Auffüllen direkt in asfreq nutzen.
# ffill füllt "nach vorne" auf, d.h. NAs werden ersetzt durch die Werte,
# die direkt vor ihnen kommen:
sea_ice.asfreq('1D', method='ffill')

In [ ]:
# bfill ist das Gegenteil von ffill
sea_ice.asfreq('1D', method='bfill')

In [ ]:
# Man kann NAs auch mit fillna ersetzen.
# Beispiel: Alle NAs durch die Zahl 10 ersetzen:
ice_daily = sea_ice.asfreq('1D')
ice_daily.head()

In [ ]:
ice_daily.fillna(10)

In [ ]:
# Etwas eleganter: Durchschnitt des Ganzen zum Auffüllen nutzen:
ice_daily.fillna(ice_daily.mean())